In [1]:
using Iterators;
using Plots
using Distributions;

In [2]:
plotly(size=(600,300)) # We choose PlotlyJS as backend because it's interactive, but you can choose any backend

Plots.PlotlyBackend()

## Helper functions

In [3]:
@everywhere function mcvar_bm(x::Vector{Float64}; batchlen::Int=100)
  nbatches = div(length(x), batchlen)
  @assert nbatches > 1 "Choose batch size such that the number of batches is greather than one"
  nbsamples = nbatches*batchlen
  batchmeans = Float64[mean(x[((j-1)*batchlen+1):(j*batchlen)]) for j = 1:nbatches]
  return batchlen*var(batchmeans)/nbsamples
end

@everywhere mcvar_iid(x) = var(x)/length(x)

@everywhere function ess(x)
  return length(x)*mcvar_iid(x)/mcvar_bm(x)
end 

## Model Definition

We consider a simple $d$ dimensional mass spring model consisting of a regular mesh of $(M+1)^d$ points in $\mathbb{R}$, with the displacement at edges $0$ and $M+1$ fixed.  The vector $(u_{\alpha})_{\alpha} \in \mathbb{R}^N$, for $N = M^d$ defines the out-of-plane displacement for the remaining points.  We assume a simple quadratic potential energy of the form
$$
    E_N(u) = \frac{1}{2}\sum_{\alpha \sim \beta} |u_\alpha  - u_{\beta}|^2 = \frac{1}{2} u\cdot H u,
$$
where $H$ is the Laplacian defined on the regular mesh $\lbrace 1 ,\ldots, M \rbrace^{d}$ with zero boundary conditions.  Given $M$ and dimension $d$, the matrix $H$ is constructed by the follwoing functions.

In [4]:
@everywhere function get_laplacian_index(p, M::Int64, d::Int64)
    return sum([p[i]*M^(i-1) for i in 1:d])
end

@everywhere function brutal_laplacian(M::Int64, d::Int64)
    dirs = speye(Int, d)
    L = zeros(M^d)

    I = Int64[]
    J = Int64[]
    V = Float64[]
    for p in product([collect(0:M-1) for k=1:d]...)
        index = get_laplacian_index(p, M, d)

        for j in 1:d
            for sign in (+1, -1)
                pnew = [p[i] + sign*dirs[i,j] for i in 1:d]
                if all(0.<= pnew .<=M-1)
                    #Calculate indices
                    new_index = get_laplacian_index(pnew, M, d)

                    #Add matrix entries
                    push!(I, index+1)
                    push!(J, new_index+1)
                    push!(V, -1)
                end
                
                push!(I, index+1)
                push!(J, index+1)            
                push!(V, 1)
            end
        end
    end
    return sparse(I,J,V)
end

In [5]:
@everywhere function generatePotentialFunctions(H)
    pot = function (x) return 0.5*dot(x, H*x) end
    gradPot = function (x) return H*x   end
    return pot, gradPot
end

## Spectrum of $H$

The spectrum of $H$ satisfies the scaling $\lambda_j \sim \frac{j^{2/d}}{M^2}$, for $j\in \mathbb{N}$.  In particular, $\sigma(H) \subset \left[\frac{K}{M^2}, K'\right]$ for $K$ and $K'$ independent of $M$.  Replotting the below images for different values of $M$ we see that the spectrum is bounded by a constant $K' \approx 12$.

In [8]:
M = 4
dim=3
H = brutal_laplacian(M,dim);
evals = eigs(H, nev=M^dim-1, which=:SM)[1]

ind = collect(1:M^dim -1)
evals_theory = (ind/(M^dim)).^(2/dim)

# plot([p1,p2])
plot(ind, [evals evals_theory],style=:auto, label=["Eigs" "Theory"]) 
yaxis!("eig_i",:log10)
xaxis!("i",:log10)
title!("Eigenvalues of H compared with predicted scaling")

In [9]:
M = 4
dim=3
H = brutal_laplacian(M,dim);
evals4 = eigs(H, nev=M^dim-1, which=:SM)[1]
ind4 = collect(1:M^dim -1)

M = 6
H = brutal_laplacian(M,dim);
evals6 = eigs(H, nev=M^dim-1, which=:SM)[1]
ind6 = collect(1:M^dim -1)

M = 8
H = brutal_laplacian(M,dim);
evals8 = eigs(H, nev=M^dim-1, which=:SM)[1]
ind8 = collect(1:M^dim -1)

M = 10
H = brutal_laplacian(M,dim);
evals10 = eigs(H, nev=M^dim-1, which=:SM)[1]
ind10 = collect(1:M^dim -1)


# plot([p1,p2])
plot()
plot!(ind4, evals4 ,style=:auto, label="M=4") 
plot!(ind6, evals6 ,style=:auto, label="M=6") 
plot!(ind8, evals8 ,style=:auto, label="M=8") 
plot!(ind10, evals10 ,style=:auto, label="M=10") 

yaxis!("eig_i",:log10)
xaxis!("i",:log10)
title!("Scaling of eigenvalues for different values of M")

In [10]:
M = 6
dim = 3;
L = brutal_laplacian(M,dim);
evals = eigs(L, nev=M^dim-1, which=:SM)[1]

ind = collect(1:M^dim -1)
evals_theory = (ind/(M^dim)).^(2/dim)

# plot([p1,p2])
plot(ind, 1.0./evals,style=:auto, label=["Covar Eigs"]) 
yaxis!("eig_i",:log10)
xaxis!("i",:log10)
title!("Spectrum of covariance operator inv(H)")

## MALA Scheme

Our objective is to compute $I = \int f(x)\mu(dx)$, where $\mu(dx) \sim e^{-\frac{1}{2}x\cdot H^{-1}x}\,dx$ and $f(x) = \frac{1}{2N}x\cdot H x$.  It is clear that 

$$I = \frac{1}{N}\frac{\int x\cdot H x e^{-\frac{x \cdot H x}{2}}\,dx}{\int e^{-\frac{x \cdot H x}{2}}\,dx} = \frac{1}{2}.$$

Moreover, the stationary variance is 
$$\frac{1}{4N^2} \frac{\int (x\cdot H x) (x\cdot H x) e^{-\frac{x \cdot H x}{2}}\,dx}{\int e^{-\frac{x \cdot H x}{2}}\,dx}  - \frac{1}{4} =  \frac{1}{2N}.$$ 

To do so we use a preconditioned Metropolis-Adjusted Langevin scheme (P-MALA), where $P$ is an appropriate preconditioning matrix.  Given the current state $x_n$ the following state is proposed:
$$
    y = x_n - P^{-1}\nabla E_N(x_n)\delta + \sqrt{2\delta}P^{-1/2}\xi_n,\quad \xi_n \sim \mathcal{N}(0, I),
$$
which is accepted with probability
$$
    \alpha(y, x_n) = \mbox{max}\left(1, \frac{\mu(y)q(x_n | y)}{\mu(x)q(y | x_n)}\right),
$$
where $q(dy | x) \sim \mathcal{N}(dy; x - \delta P^{-1}\nabla E_n(x), 2\delta P^{-1})$.  When $P = I$ the scheme reduces to the standard MALA scheme.

We shall consider the cases when $P=I$ (and scheme reduces to MALA) versus the case $P= H$.

In [11]:
@everywhere function timestepPMALA(x0::Array{Float64, 1}, P::SparseMatrixCSC{Float64,Int64}, 
                                   numsteps::Int64, latticeSize::Int64, 
                                   dim::Int64, delta::Float64; prec=true)
    num_acc = 0
    effDim = latticeSize^dim
    
    state = zeros(effDim, numsteps)
    state[:, 1] = x0
        
    V, gradV = generatePotentialFunctions(P)
    
    if !prec
        P = speye(effDim)
    end
    Pfact = cholfact(full(P))
    
    Vx = V(x0)
    driftx = Pfact\gradV(x0)
    
    @inbounds @fastmath for n=1:numsteps-1
        x = state[:, n]
        
        noise = Pfact[:U]\randn(effDim)
        y = x - driftx*delta + sqrt(2*delta)*noise
        
        r = y - (x - delta*driftx)
        lyx = dot(r, P*r)/(4*delta)
                
        drifty = Pfact\gradV(y)
        Vy = V(y)
        r = x - (y - delta*drifty)
        lxy = dot(r, P*r)/(4*delta)
        
        if -log(rand()) > V(y) - V(x) + lxy - lyx
            x = y
            num_acc += 1
            Vx = Vy
            driftx = drifty
        end 
        
        state[:,n+1] = x
    end
    
    return state, num_acc/numsteps
end

Since we consider a quadratic potential we can sample from the invariant distribution $\mu(dx)$ directly, using the following function:

In [12]:
@everywhere function sampleStatDist(Hinv::Array{Float64,2})
    return rand(MvNormal(Hinv))
end

The following function generates a timeseries using P-MALA and applies the function $f(x) = \frac{1}{N}\frac{1}{2}x\cdot H x$, starting from stationarity.

In [13]:
@everywhere function estfun(M::Int64, dim::Int64, H::AbstractSparseMatrix, Hinv, numsteps::Int64,  delta::Float64, prec::Bool)
   # println("M = $M, dim = $dim, numsteps = $numsteps, delta = $delta")
    x0 = sampleStatDist(Hinv)
    x_trace, acc_rate=  timestepPMALA(x0, H, numsteps, M, dim, delta; prec=prec);
    
    V,_ = generatePotentialFunctions(H);

    return vec(mapslices(V, x_trace, 1)/(M^dim))
end


The following function implements a basic grid search approach to tuning step-size based on Effective Sample Size.  This is not a very robust approach.

In [14]:
function tune_stepsize(dim::Int64, numsteps::Int64, prec::Bool, deltas, Ms)
    delta_opt = zeros(length(Ms))

    for (i, M) in enumerate(Ms)
        H = brutal_laplacian(M, dim);
        Hinv= inv(full(H))
        ess_vs_delta = Array{Float64,1}([ess(estfun(M, dim, H, Hinv, numsteps,  delta, prec)) for delta in deltas])
        ess_vs_delta[ess_vs_delta.==Inf]=0
        ess_max, index = findmax(ess_vs_delta);
        delta_max = deltas[index];
        delta_opt[i] = delta_max
        println("[Tuning] M = $M : $ess_max, $index, $delta_max")
    end
    
    return delta_opt
end

tune_stepsize (generic function with 1 method)

Given a sequence of lattice sizes $M$ and optimal time-step sizes, computes variance and MSE of runs for $N_{runs}$ independent realisations of $Numsteps$ step chains starting from the invariant measure.

In [15]:
function compute_trace_stats(Ms, delta_opts, Nruns, Numsteps, prec)
    vars = zeros(length(Ms))
    mses = zeros(length(Ms))

    for (i, M) in enumerate(Ms)
        delta = delta_opts[i]
        H = brutal_laplacian(M, dim);
        Hinv = inv(full(H))
        runs = Array{Float64}([mean(estfun(M, dim, H, Hinv, numsteps,  delta, prec)) for j in 1:Nruns])
        vars[i] = var(runs)
        mses[i] = mean((runs-0.5).^2)
        println("[Computing Stats] M = $M,  : Var = ", vars[i], " MSE = ", mses[i])
    end
    
    return vars, mses
end

compute_trace_stats (generic function with 1 method)

# Variance and MSE computations

In this section we compare the variance and MSE for the preconditioned MALA with $P = H$ to the standard MALA approach, where $P = I$.  We compare in terms of dimension $d$ and lattice size $M$.

In [16]:
deltaP = 0.01:0.1:0.6
deltaU = 0.01:0.01:0.1

numsteps=10000
Nruns = 100
Ms = [2, 4, 6, 8, 10];

## One dimension

In [34]:
dim=1

#Preconditioned 
delta_opts = tune_stepsize(dim, numsteps, true, deltaP, Ms);
vars1_pmala, mses1_pmala = compute_trace_stats(Ms, delta_opts, Nruns, numsteps, true);

#NonPreconditioned 
delta_opts = tune_stepsize(dim, numsteps, false, deltaU, Ms);
vars1_mala, mses1_mala = compute_trace_stats(Ms, delta_opts, Nruns, numsteps, false);


[Tuning] M = 2 : 4577.702290530713, 6, 0.51
[Tuning] M = 4 : 4559.320115130788, 5, 0.41000000000000003
[Tuning] M = 6 : 3831.5374912697944, 5, 0.41000000000000003
[Tuning] M = 8 : 4048.9541724107294, 5, 0.41000000000000003
[Tuning] M = 10 : 3531.8262659131033, 6, 0.51
[Computing Stats] M = 2,  : Var = 4.743683410455971e-5 MSE = 4.811596763744414e-5
[Computing Stats] M = 4,  : Var = 3.243453956428163e-5 MSE = 3.2142597218578585e-5
[Computing Stats] M = 6,  : Var = 2.2150109054378778e-5 MSE = 2.211468713091029e-5
[Computing Stats] M = 8,  : Var = 1.3318615064556813e-5 MSE = 1.3218430194525664e-5
[Computing Stats] M = 10,  : Var = 1.6865991292615293e-5 MSE = 1.6698749104207276e-5
[Tuning] M = 2 : 1969.1958645456914, 9, 0.09
[Tuning] M = 4 : 1318.0629763604857, 8, 0.08
[Tuning] M = 6 : 960.1930185932819, 10, 0.1
[Tuning] M = 8 : 1137.48106486452, 10, 0.1
[Tuning] M = 10 : 948.1923360561749, 8, 0.08
[Computing Stats] M = 2,  : Var = 0.00018590134696824809 MSE = 0.00018407556468046936
[Compu

Plotting the results here

In [18]:
plot(Ms, [vars1_pmala vars1_mala],style=:auto, label=["P-MALA" "MALA"]) 
yaxis!("Var",:log10)
xaxis!("M",:log10)
title!("Variance for N=$numsteps timesteps for d =$dim")

## Two dimensions

In [36]:
dim=2

#Preconditioned 
delta_opts = tune_stepsize(dim, numsteps, true, deltaP, Ms);
vars2_pmala, mses2_pmala = compute_trace_stats(Ms, delta_opts, Nruns, numsteps, true);

#NonPreconditioned 
delta_opts = tune_stepsize(dim, numsteps, false, deltaU, Ms);
vars2_mala, mses2_mala = compute_trace_stats(Ms, delta_opts, Nruns, numsteps, false);


[Tuning] M = 2 : 3929.2406216215322, 6, 0.51
[Tuning] M = 4 : 2524.184110351526, 5, 0.41000000000000003
[Tuning] M = 6 : 2247.443491070385, 4, 0.31000000000000005
[Tuning] M = 8 : 1769.3462423577384, 3, 0.21000000000000002
[Tuning] M = 10 : 1170.2639508430175, 3, 0.21000000000000002
[Computing Stats] M = 2,  : Var = 2.9057354916827298e-5 MSE = 2.8768233666509867e-5
[Computing Stats] M = 4,  : Var = 1.2971929534270575e-5 MSE = 1.3490048276055152e-5
[Computing Stats] M = 6,  : Var = 5.971520219300189e-6 MSE = 6.028778937112851e-6
[Computing Stats] M = 8,  : Var = 6.05693778376e-6 MSE = 6.087699184131924e-6
[Computing Stats] M = 10,  : Var = 4.343361770190897e-6 MSE = 4.299956273872559e-6
[Tuning] M = 2 : 4855.210168210832, 10, 0.1
[Tuning] M = 4 : 2305.702219328806, 8, 0.08
[Tuning] M = 6 : 1262.0146899895285, 6, 0.06
[Tuning] M = 8 : 1023.9607391751476, 7, 0.07
[Tuning] M = 10 : 999.4045455955932, 5, 0.05
[Computing Stats] M = 2,  : Var = 3.86083227721067e-5 MSE = 4.275419481886445e-5
[

In [19]:
plot(Ms, [vars2_pmala vars2_mala],style=:auto, label=["P-MALA" "MALA"]) 
yaxis!("Var",:log10)
xaxis!("M",:log10)
title!("Variance for N=$numsteps timesteps for d =$dim")

## Three dimensions

In [38]:
dim=3

#Preconditioned 
delta_opts = tune_stepsize(dim, numsteps, true, deltaP, Ms);
vars3_pmala, mses3_pmala = compute_trace_stats(Ms, delta_opts, Nruns, numsteps, true);

#NonPreconditioned 
delta_opts = tune_stepsize(dim, numsteps, false, deltaU, Ms);
vars3_mala, mses3_mala = compute_trace_stats(Ms, delta_opts, Nruns, numsteps, false);

[Tuning] M = 2 : 4429.815581634926, 6, 0.51
[Tuning] M = 4 : 1712.945444633848, 3, 0.21000000000000002
[Tuning] M = 6 : 1031.8445868367066, 3, 0.21000000000000002
[Tuning] M = 8 : 621.730146692421, 2, 0.11
[Tuning] M = 10 : 891.089108910891, 6, 0.51
[Computing Stats] M = 2,  : Var = 1.9041155692764443e-5 MSE = 1.894347047590746e-5
[Computing Stats] M = 4,  : Var = 5.403423953182649e-6 MSE = 5.388428018946792e-6
[Computing Stats] M = 6,  : Var = 2.975427378368099e-6 MSE = 2.968678359033218e-6
[Computing Stats] M = 8,  : Var = 1.1484964877000937e-6 MSE = 1.1696908211731702e-6
[Computing Stats] M = 10,  : Var = 0.000516977828963673 MSE = 0.0005119184142096743
[Tuning] M = 2 : 3306.608542208796, 9, 0.09
[Tuning] M = 4 : 1254.860122701526, 5, 0.05
[Tuning] M = 6 : 755.0163993850376, 2, 0.02
[Tuning] M = 8 : 1212.8712871287128, 8, 0.08
[Tuning] M = 10 : 542.4847629746399, 2, 0.02
[Computing Stats] M = 2,  : Var = 2.3927449598684965e-5 MSE = 2.4008911968531133e-5
[Computing Stats] M = 4,  : V

In [20]:
plot(Ms, [vars3_pmala vars3_mala],style=:auto, label=["P-MALA" "MALA"]) 
yaxis!("Var",:log10)
xaxis!("M",:log10)
title!("Variance for N=$numsteps timesteps for d =$dim")

I would not read too much into the last two data points, most likely poor choice of tuning parameter.